In [11]:
from pathlib import Path
import pandas as pd
from carps.analysis.gather_data import normalize_logs, get_interpolated_performance_df, load_logs, process_logs
from carps.analysis.utils import filter_only_final_performance

def load_set(paths: list[str], set_id: str = "unknown") -> tuple[pd.DataFrame, pd.DataFrame]:
    logs = []
    for p in paths:
        fn = Path(p) / "trajectory.parquet"
        if not fn.is_file():
            fn = Path(p) / "logs.parquet"
        logs.append(pd.read_parquet(fn))

    df = pd.concat(logs).reset_index(drop=True)
    df_cfg = pd.concat([pd.read_parquet(Path(p) / "logs_cfg.parquet") for p in paths]).reset_index(drop=True)
    df["set"] = set_id
    return df, df_cfg

scenario = "blackbox"
set_id = "full"
subset_size = 30
paths = ["../runs/SMAC3-BlackBoxFacade", "../runs/RandomSearch", "../runs/Nevergrad-CMA-ES"]

scenario = "multi-fidelity-objective"
set_id = "full"
subset_size = 9
paths = ["../runs_MOMF/SMAC3-MOMF-GP", "../runs_MOMF/RandomSearch", "../runs_MOMF/Nevergrad-DE"]

# scenario = "multi-objective"
# set_id = "full"
# subset_size = 10
# paths = ["../runs_MO/Optuna-MO", "../runs_MO/RandomSearch", "../runs_MO/Nevergrad-DE"]

# scenario = "multi-fidelity"
# set_id = "full"
# subset_size = 20
# paths = ["../runs/SMAC3-Hyperband", "../runs/SMAC3-MultiFidelityFacade", "../runs_/DEHB"]

fn = f"../data/{scenario}_{set_id}_logs.parquet"
if Path(fn).is_file():
    df = pd.read_parquet(fn)
else:
    D = []
    for rundir in paths:
        df, df_cfg = load_set([rundir], set_id="full")
        D.append(df)

    df = pd.concat(D).reset_index(drop=True)
    del D

    df = normalize_logs(df)

    normalize_performance = False
    perf_col = "trial_value__cost_inc_norm" if normalize_performance else "trial_value__cost_inc"

    df.to_parquet(f"../data/{scenario}_{set_id}_logs.parquet")
df = filter_only_final_performance(df=df)

FileNotFoundError: [Errno 2] No such file or directory: '../runs_MOMF/SMAC3-MOMF-GP/logs.parquet'

In [ ]:
subset_size = 9
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit
from carps.utils.pareto_front import pareto

problem_ids = df["problem_id"].unique()
# print(problem_ids)
benchmark_ids = [pid.split("/")[0] for pid in problem_ids]
n_problems = len(problem_ids)

print(n_problems)

n_splits = n_problems // (subset_size * 2)
print(n_splits)
X = problem_ids
y = benchmark_ids

seeds = np.arange(0, 5)

split_classes = [StratifiedShuffleSplit, ShuffleSplit]

new_subset_performance = []


for split_class in split_classes:
    for seed in seeds:
        sss = split_class(n_splits=n_splits, test_size=0.5, random_state=seed)
        sss.get_n_splits(X, y)
        for i, (train_index, test_index) in enumerate(sss.split(X, y)):
            # print(split_class, seed, i)

            ids_dev = problem_ids[train_index]
            ids_test = problem_ids[test_index]
            for _set_id, ids in zip(["dev", "test"], (ids_dev, ids_test)):
                new_df = df[df["problem_id"].isin(ids)].copy()
                set_id = f"split_{i}_seed_{seed}_subset_{_set_id}"
                print(set_id)
                new_df["set"] = set_id
                new_subset_performance.append(new_df)

df_new = pd.concat(new_subset_performance).reset_index(drop=True)

121
6
split_0_seed_0_subset_dev
split_0_seed_0_subset_test
split_1_seed_0_subset_dev
split_1_seed_0_subset_test
split_2_seed_0_subset_dev
split_2_seed_0_subset_test
split_3_seed_0_subset_dev
split_3_seed_0_subset_test
split_4_seed_0_subset_dev
split_4_seed_0_subset_test
split_5_seed_0_subset_dev
split_5_seed_0_subset_test
split_0_seed_1_subset_dev
split_0_seed_1_subset_test
split_1_seed_1_subset_dev
split_1_seed_1_subset_test
split_2_seed_1_subset_dev
split_2_seed_1_subset_test
split_3_seed_1_subset_dev
split_3_seed_1_subset_test
split_4_seed_1_subset_dev
split_4_seed_1_subset_test
split_5_seed_1_subset_dev
split_5_seed_1_subset_test
split_0_seed_2_subset_dev
split_0_seed_2_subset_test
split_1_seed_2_subset_dev
split_1_seed_2_subset_test
split_2_seed_2_subset_dev
split_2_seed_2_subset_test
split_3_seed_2_subset_dev
split_3_seed_2_subset_test
split_4_seed_2_subset_dev
split_4_seed_2_subset_test
split_5_seed_2_subset_dev
split_5_seed_2_subset_test
split_0_seed_3_subset_dev
split_0_seed_3

In [ ]:
# _df = pd.concat([df, df_new]).reset_index(drop=True)
_df = df_new

In [ ]:
from autorank._util import get_sorted_rank_groups, RankResult
from carps.analysis.plot_ranking import calc_critical_difference

rank_results = {}
for (scenario, set_id), gdf in _df.groupby(by=["scenario", "set"]):
    perf_col: str = "trial_value__cost_inc_norm"
    identifier = f"{scenario}_{set_id}"
    result = calc_critical_difference(gdf, identifier=identifier, figsize=(8, 3), perf_col=perf_col, plot_diagram=False)
    sorted_ranks, names, groups = get_sorted_rank_groups(result, reverse=False)
    rank_results[(scenario, set_id)] = result

[19:02:06] INFO     Lost following experiments: optimizer_id                      Nevergrad-DE  ]8;id=534370;file:///home/numina/Documents/repos/CARP-S-Experiments/lib/CARP-S/carps/analysis/run_autorank.py\run_autorank.py]8;;\:]8;id=984210;file:///home/numina/Documents/repos/CARP-S-Experiments/lib/CARP-S/carps/analysis/run_autorank.py#113\113]8;;\
                    Optuna-MO  RandomSearch                                                                        
                    problem_id                                                                                     
                    yahpo/mo/iaml_ranger/1067/None        0.336826        NaN      0.217303                        
                    yahpo/mo/iaml_ranger/1489/None        0.174604        NaN      0.004866                        
                    yahpo/mo/iaml_super/1067/None              NaN        NaN      0.000075                        
                    yahpo/mo/iaml_super/1489/None              NaN        NaN      0.000240                        
                    yahpo/mo/iaml_xgboost/40981/None      0.201979        NaN      0.001476                        
                    yahpo/mo/rbv2_ranger/375/None         0.218806        NaN      0.049281                        
                    yahpo/mo/rbv2_ranger/40979/None       0.197883        NaN      0.067001                        
                    yahpo/mo/rbv2_ranger/6/None           0.110805        NaN      0.017989                        
                    yahpo/mo/rbv2_super/1053/None              NaN        NaN      0.005830                        
                    yahpo/mo/rbv2_super/1457/None              NaN        NaN      0.000074                        
                    yahpo/mo/rbv2_xgboost/12/None         0.040141        NaN      0.000009                        
                    yahpo/mo/rbv2_xgboost/28/None         0.007840        NaN      0.000001                        

Rejecting null hypothesis that data is normal for column Nevergrad-DE (p=0.000000<0.016667)
Rejecting null hypothesis that data is normal for column Optuna-MO (p=0.000000<0.016667)
Rejecting null hypothesis that data is normal for column RandomSearch (p=0.000000<0.016667)
Using Friedman test as omnibus test
Rejecting null hypothesis that there is no difference between the distributions (p=0.000042)
Using Nemenyi post-hoc test. Differences are significant,if the distance between the mean ranks is greater than the critical distance.
              meanrank    median       mad  ci_lower  ci_upper effect_size  \
Nevergrad-DE  1.633333  0.007830  0.007803  0.001327  0.044524         0.0   
Optuna-MO     2.077778  0.009641  0.009613  0.002813  0.058713   -0.139513   
RandomSearch  2.288889  0.011237   0.01114  0.002854  0.051639   -0.238931   

               magnitude  
Nevergrad-DE  negligible  
Optuna-MO     negligible  
RandomSearch       small  


/home/numina/micromamba/envs/carpsexp/lib/python3.11/site-packages/autorank/_util.py:432: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if abs(sorted_ranks[i] - sorted_ranks[j]) <= critical_difference:


[19:02:08] INFO     Lost following experiments: optimizer_id                      Nevergrad-DE  ]8;id=620702;file:///home/numina/Documents/repos/CARP-S-Experiments/lib/CARP-S/carps/analysis/run_autorank.py\run_autorank.py]8;;\:]8;id=961564;file:///home/numina/Documents/repos/CARP-S-Experiments/lib/CARP-S/carps/analysis/run_autorank.py#113\113]8;;\
                    Optuna-MO  RandomSearch                                                                        
                    problem_id                                                                                     
                    yahpo/mo/iaml_ranger/1489/None        0.174604        NaN      0.004866                        
                    yahpo/mo/iaml_xgboost/1489/None       0.203513        NaN      0.000394                        
                    yahpo/mo/iaml_xgboost/40981/None      0.201979        NaN      0.001476                        
                    yahpo/mo/rbv2_ranger/375/None         0.218806        NaN      0.049281                        
                    yahpo/mo/rbv2_ranger/40979/None       0.197883        NaN      0.067001                        
                    yahpo/mo/rbv2_super/1457/None              NaN        NaN      0.000074                        
                    yahpo/mo/rbv2_super/6/None                 NaN        NaN      0.000088                        
                    yahpo/mo/rbv2_xgboost/182/None        0.010498        NaN      0.000003                        

Rejecting null hypothesis that data is normal for column Nevergrad-DE (p=0.000000<0.016667)
Rejecting null hypothesis that data is normal for column Optuna-MO (p=0.000000<0.016667)
Rejecting null hypothesis that data is normal for column RandomSearch (p=0.000000<0.016667)
Using Friedman test as omnibus test
Rejecting null hypothesis that there is no difference between the distributions (p=0.000098)
Using Nemenyi post-hoc test. Differences are significant,if the distance between the mean ranks is greater than the critical distance.
              meanrank    median       mad  ci_lower  ci_upper effect_size  \
Nevergrad-DE  1.673684  0.008228  0.008137  0.003283  0.046377         0.0   
Optuna-MO     2.031579  0.009992  0.009963  0.004852  0.053638   -0.130817   
RandomSearch  2.294737  0.011273  0.011179  0.006476  0.060899   -0.210063   

               magnitude  
Nevergrad-DE  negligible  
Optuna-MO     negligible  
RandomSearch       small  


...

Rejecting null hypothesis that data is normal for column Nevergrad-DE (p=0.000000<0.016667)
Rejecting null hypothesis that data is normal for column Optuna-MO (p=0.000000<0.016667)
Rejecting null hypothesis that data is normal for column RandomSearch (p=0.000000<0.016667)
Using Friedman test as omnibus test
Rejecting null hypothesis that there is no difference between the distributions (p=0.003504)
Using Nemenyi post-hoc test. Differences are significant,if the distance between the mean ranks is greater than the critical distance.
              meanrank    median       mad  ci_lower  ci_upper effect_size  \
Nevergrad-DE  1.807692  0.008578  0.008321  0.003283  0.064612         0.0   
Optuna-MO     1.884615  0.009641  0.009576  0.004593  0.053638   -0.079914   
RandomSearch  2.307692  0.013190  0.012591  0.007458   0.07337   -0.291523   

               magnitude  
Nevergrad-DE  negligible  
Optuna-MO     negligible  
RandomSearch       small  


/home/numina/micromamba/envs/carpsexp/lib/python3.11/site-packages/autorank/_util.py:432: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if abs(sorted_ranks[i] - sorted_ranks[j]) <= critical_difference:


[19:03:18] INFO     Lost following experiments: optimizer_id                      Nevergrad-DE  ]8;id=485449;file:///home/numina/Documents/repos/CARP-S-Experiments/lib/CARP-S/carps/analysis/run_autorank.py\run_autorank.py]8;;\:]8;id=457927;file:///home/numina/Documents/repos/CARP-S-Experiments/lib/CARP-S/carps/analysis/run_autorank.py#113\113]8;;\
                    Optuna-MO  RandomSearch                                                                        
                    problem_id                                                                                     
                    yahpo/mo/iaml_ranger/1067/None        0.336826        NaN      0.217303                        
                    yahpo/mo/iaml_ranger/1489/None        0.174604        NaN      0.004866                        
                    yahpo/mo/iaml_super/1067/None              NaN        NaN      0.000075                        
                    yahpo/mo/iaml_xgboost/40981/None      0.201979        NaN      0.001476                        
                    yahpo/mo/rbv2_ranger/375/None         0.218806        NaN      0.049281                        
                    yahpo/mo/rbv2_ranger/6/None           0.110805        NaN      0.017989                        
                    yahpo/mo/rbv2_super/1053/None              NaN        NaN      0.005830                        
                    yahpo/mo/rbv2_super/1457/None              NaN        NaN      0.000074                        
                    yahpo/mo/rbv2_super/6/None                 NaN        NaN      0.000088                        
                    yahpo/mo/rbv2_xgboost/12/None         0.040141        NaN      0.000009                        
                    yahpo/mo/rbv2_xgboost/182/None        0.010498        NaN      0.000003                        
                    yahpo/mo/rbv2_xgboost/28/None         0.007840        NaN      0.000001                        

Rejecting null hypothesis that data is normal for column Nevergrad-DE (p=0.000000<0.016667)
Rejecting null hypothesis that data is normal for column Optuna-MO (p=0.000000<0.016667)
Rejecting null hypothesis that data is normal for column RandomSearch (p=0.000000<0.016667)
Using Friedman test as omnibus test
Rejecting null hypothesis that there is no difference between the distributions (p=0.000127)
Using Nemenyi post-hoc test. Differences are significant,if the distance between the mean ranks is greater than the critical distance.
              meanrank    median       mad  ci_lower  ci_upper effect_size  \
Nevergrad-DE  1.645570  0.007503  0.007412  0.001258   0.03076         0.0   
Optuna-MO     2.037975  0.008966  0.008935  0.002813  0.053638   -0.120181   
RandomSearch  2.316456  0.010683  0.010496  0.003702  0.051639   -0.236084   

               magnitude  
Nevergrad-DE  negligible  
Optuna-MO     negligible  
RandomSearch       small  


/home/numina/micromamba/envs/carpsexp/lib/python3.11/site-packages/autorank/_util.py:432: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if abs(sorted_ranks[i] - sorted_ranks[j]) <= critical_difference:


In [ ]:
R = []
for i, (k, v) in enumerate(rank_results.items()):
    d = pd.DataFrame({
        "scenario": k[0],
        "set": k[1],
        **v.rankdf["meanrank"]
    }, index=[i]
    ).melt(id_vars=["scenario", "set"], var_name="optimizer_id", value_name="meanrank")
    d["order"] = d.rank(method="max", numeric_only=True).astype(int)
    R.append(d)
    # break

df_rank = pd.concat(R).reset_index(drop=True)
df_rank = df_rank[df_rank["set"] != "full"]
def rename(x: str) -> str:
    if x in ["dev", "test"]:
        return f"discrepancy_{x}"
    return x
df_rank["set"] = df_rank["set"].apply(rename)
df_rank

,scenario,set,optimizer_id,meanrank,order
0,multi-objective,split_0_seed_0_subset_dev,Nevergrad-DE,1.633333,1
1,multi-objective,split_0_seed_0_subset_dev,Optuna-MO,2.077778,2
2,multi-objective,split_0_seed_0_subset_dev,RandomSearch,2.288889,3
3,multi-objective,split_0_seed_0_subset_test,Nevergrad-DE,1.673684,1
4,multi-objective,split_0_seed_0_subset_test,Optuna-MO,2.031579,2
...,...,...,...,...,...
175,multi-objective,split_5_seed_4_subset_dev,Optuna-MO,1.884615,2
176,multi-objective,split_5_seed_4_subset_dev,RandomSearch,2.307692,3
177,multi-objective,split_5_seed_4_subset_test,Nevergrad-DE,1.645570,1
178,multi-objective,split_5_seed_4_subset_test,Optuna-MO,2.037975,2


In [ ]:

original_optimizers = {
    "blackbox": ["RandomSearch", "SMAC3-BlackBoxFacade", "Nevergrad-CMA-ES"],
    "multi-objective": ["RandomSearch", "Optuna-MO", "Nevergrad-DE"],
    "multi-fidelity": ["SMAC3-Hyperband", "SMAC3-MultiFidelityFacade", "DEHB"],
    "multi-fidelity-objective": ["RandomSearch", "SMAC3-MOMF-GP", "Nevergrad-DE"],
}


is_different = {}
for scenario, gdf in df_rank.groupby(by="scenario"):
    set_origins = gdf["set"].apply(lambda x: "_".join(x.split("_")[:-2])).unique()   
    for set_origin in set_origins:
        origs = original_optimizers[scenario]
        df_dev = gdf[gdf["set"] == f"{set_origin}_subset_dev"]
        df_test = gdf[gdf["set"] == f"{set_origin}_subset_test"]
        order_dev = []
        order_test = []
        for orig in origs:
            order_dev.append(df_dev[df_dev["optimizer_id"] == orig]["order"].values[0])
            order_test.append(df_test[df_test["optimizer_id"] == orig]["order"].values[0])
        _is_different = order_dev != order_test
        is_different[(scenario, set_origin)] = _is_different
is_different

{('multi-objective', 'split_0_seed_0'): False,
 ('multi-objective', 'split_0_seed_1'): False,
 ('multi-objective', 'split_0_seed_2'): False,
 ('multi-objective', 'split_0_seed_3'): False,
 ('multi-objective', 'split_0_seed_4'): False,
 ('multi-objective', 'split_1_seed_0'): False,
 ('multi-objective', 'split_1_seed_1'): False,
 ('multi-objective', 'split_1_seed_2'): False,
 ('multi-objective', 'split_1_seed_3'): False,
 ('multi-objective', 'split_1_seed_4'): False,
 ('multi-objective', 'split_2_seed_0'): False,
 ('multi-objective', 'split_2_seed_1'): False,
 ('multi-objective', 'split_2_seed_2'): False,
 ('multi-objective', 'split_2_seed_3'): False,
 ('multi-objective', 'split_2_seed_4'): False,
 ('multi-objective', 'split_3_seed_0'): False,
 ('multi-objective', 'split_3_seed_1'): False,
 ('multi-objective', 'split_3_seed_2'): False,
 ('multi-objective', 'split_3_seed_3'): False,
 ('multi-objective', 'split_3_seed_4'): False,
 ('multi-objective', 'split_4_seed_0'): False,
 ('multi-obje

In [ ]:
import pandas as pd
import numpy as np



fn_template = "ranks_per_set_{scenario}.csv"

decimal_places = 2

final_str = r"""
\begin{{table}}[h]
    \caption{{{caption}}}
    \label{{{label}}}
    \centering
    %\resizebox{{0.4\textwidth}}{{!}}{{
    {table_string}
    %}}
\end{{table}}
"""

float_format = lambda x: ("{:0." + str(decimal_places) + "f}").format(x) if not np.isnan(x) else "-"


for scenario, gdf in df_rank.groupby("scenario"):
    fn = fn_template.format(scenario=scenario)
    sorter = gdf[gdf["set"]=="dev"].sort_values("meanrank")["optimizer_id"].to_list()

    R = gdf.pivot_table(index="set", columns="optimizer_id", values="order").map(int)
    origs = original_optimizers[scenario]
    origs.sort(key=lambda x: sorter.index(x))
    cols = origs + [c for c in R.columns if c not in original_optimizers[scenario]]
    R = R[cols]


    MR = gdf.pivot_table(index="set", columns="optimizer_id", values="meanrank").map(lambda x: f"{x:.2f}" if not isinstance(x, str) else x)
    MR = MR[cols]
    for i, ((idx, row), (idx2, row2)) in enumerate(zip(MR.iterrows(), R.iterrows())):
        for j in range(len(row)):
            row.iloc[j] = row.iloc[j] + f" ({int(row2.iloc[j])})"

    print(MR)
    # table_str = MR.to_latex(float_format=float_format, na_rep="-").strip()
    # caption = f"Mean Ranking for Scenario {scenario}"
    # label = f"tab:ranking_validation_{scenario}"
    # table_str = final_str.format(table_string=table_str, label=label, caption=caption)
    # table_str = table_str.replace("_", "\_")

    # with open(fn + ".tex", "w") as file:
    #     file.write(table_str)
    # print(table_str)

ValueError: 'RandomSearch' is not in list

In [ ]:
# from pathlib import Path
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit
# from carps.utils.pareto_front import pareto

# def read_set(fn: Path) -> pd.DataFrame:
#     df = pd.read_csv(fn)
#     df["problem_id"] = df["problem_id"].apply(lambda x: "bbob/" + x if x.startswith("noiseless") else x)
#     df["benchmark_id"] = df["problem_id"].apply(lambda x: x.split("/")[0])
#     df = df.melt(id_vars=["problem_id", "benchmark_id"], value_vars=p_cols, var_name="optimizer_id", value_name="performance")    
#     return df

# performance_fn = Path("/scratch/hpc-prf-intexml/cbenjamins/repos/CARP-S-Experiments/lib/CARP-S/subselection/data/BB/default/df_crit.csv")
# performance = pd.read_csv(performance_fn)
# performance["problem_id"] = performance["problem_id"].apply(lambda x: "bbob/" + x if x.startswith("noiseless") else x)
# problem_ids = list(performance["problem_id"])
# n_problems = len(problem_ids)
# subset_size = 30

# rankings = []

# performance["benchmark_id"] = performance["problem_id"].apply(lambda x: x.split("/")[0])
# p_cols = [c for c in performance.columns if c not in ["problem_id", "benchmark_id"]]
# performance = performance.melt(id_vars=["problem_id", "benchmark_id"], value_vars=p_cols, var_name="optimizer_id", value_name="performance")


# path_subset_dev = performance_fn.parent / f"subset_{subset_size}.csv"
# subset_dev = read_set(path_subset_dev)

# path_subset_test = performance_fn.parent / f"subset_complement_subset_{subset_size}.csv"
# subset_test = read_set(path_subset_test)

# problem_ids_dev = subset_dev["problem_id"].to_list()
# problem_ids_test = subset_test["problem_id"].to_list()

# seeds = np.arange(0, 10)
# performance["benchmark_id"] = performance["problem_id"].apply(lambda x: x.split("/")[0])
# benchmark_ids = list(performance["benchmark_id"].unique())


# def add_rank(df: pd.DataFrame) -> pd.DataFrame:
#     D = []
#     for problem_id, gdf in df.groupby(["problem_id"]):
#         pivot = gdf.pivot(index=["problem_id", "benchmark_id"], columns="optimizer_id", values="performance").reset_index()
#         if not all([p in pivot.columns for p in p_cols]):
#             print(gdf.columns)
#             continue
#         rank = pivot[p_cols].rank(axis=1).melt(value_vars=p_cols, var_name="optimizer_id", value_name="rank")
#         gdf = pd.merge(gdf, rank, on=["optimizer_id"])
#         D.append(gdf)
#     D = pd.concat(D).reset_index(drop=True)
#     return D

# def get_ranking(df: pd.DataFrame, set_id: str, split: int = -1, seed: int = -1) -> pd.DataFrame:
#     df = add_rank(df)
#     df["set"] = set_id
#     df["split"] = split
#     df["seed"] = seed
#     return df


# rankings = []

# aggregate_func = np.mean

# mean_ranking_full = get_ranking(performance, "full")
# rankings.append(mean_ranking_full)

# mean_ranking_dev = get_ranking(subset_dev, "discrepancy_dev")
# rankings.append(mean_ranking_dev)

# mean_ranking_test = get_ranking(subset_test, "discrepancy_test")
# rankings.append(mean_ranking_test)  

# n_splits = n_problems // (subset_size * 2)
# print(n_splits)
# X = performance.values
# y = performance["benchmark_id"].to_numpy()

# split_classes = [StratifiedShuffleSplit, ShuffleSplit]

# # for split_class in split_classes:
# #     for seed in seeds:
# #         sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.5, random_state=seed)
# #         sss.get_n_splits(X, y)
# #         for i, (train_index, test_index) in enumerate(sss.split(X, y)):
# #             D_dev = performance.iloc[train_index]
# #             D_dev = get_ranking(D_dev, f"{split_class.__name__}_dev", i, seed)
# #             rankings.append(D_dev)

# #             D_test = performance.iloc[test_index]
# #             D_test = get_ranking(D_dev, f"{split_class.__name__}_dev", i, seed)
# #             rankings.append(D_test)


# df_ranking = pd.concat(rankings).reset_index(drop=True)

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# sns.set_style("whitegrid")
# sns.set_palette("colorblind")

# gdf = df_ranking
# fig = plt.figure(figsize=(10,4))
# ax = fig.add_subplot(111)
# ax = sns.violinplot(data=gdf, x="optimizer_id", y="rank", hue="set", cut=0, ax=ax)
# # ax.set_title(f"use_mean: {gid}")
# plt.show()

In [ ]:
# gdf = df_ranking
# fig = plt.figure(figsize=(10,4))
# ax = fig.add_subplot(111)
# ax = sns.histplot(data=gdf, x="performance", hue="optimizer_id", ax=ax,  element="step")
# ax = fig.add_subplot(211)
# ax = sns.histplot(data=gdf, x="rank", hue="optimizer_id", ax=ax,  element="step")
# # ax.set_title(f"use_mean: {gid}")
# plt.show()

In [ ]:
# def distance_bhattacharaya(p, q):
#     return -np.log(np.sum(np.sqrt(p*q)))

# def distance_kullback_leibler(p, q):
#     return np.sum(p * np.log(p / q))

# # Performance
# distances = {}
# bin_edges = None
# for gid, gdf in df_ranking.groupby(["optimizer_id", "set"]):
#     values = gdf["performance"].values
#     q = np.histogram(values, bins=20, range=[0,1], density=True)
#     bin_edges = q[1]
#     distances[gid] = q[0]

# distributions = pd.DataFrame(distances).T
# distributions.index.name = ("optimizer_id", "set")
# distributions = distributions.reset_index(names=["optimizer_id", "set"])
# # print(distributions.columns)

# dist = {}
# for gid, gdf in distributions.groupby(by=["optimizer_id", "set"]):
#     for gid2, gdf2 in distributions.groupby(by=["optimizer_id", "set"]):
#         if gid[0] != gid2[0]:
#             continue
#         p = gdf.iloc[0].values[2:].astype(float)
#         q = gdf2.iloc[0].values[2:].astype(float)
#         d = distance_bhattacharaya(p, q)
#         # print(f"{gid} vs {gid2}: {d}")
#         dist[(gid, gid2)] = d

# heatmaps = {}
# n_sets = distributions["set"].nunique()
# sets = distributions["set"].unique()
# for ((opt, set_id), (opt, set_id2)), distance in dist.items():
#     # print(f"{opt} {set_id} vs {set_id2}: {distance}") 
#     if not opt in heatmaps:
#         heatmaps[opt] = np.zeros((n_sets, n_sets))
#     heatmaps[opt][np.where(sets == set_id)[0][0], np.where(sets == set_id2)[0][0]] = distance

# fig = plt.figure(figsize=(12, 4))
# axes = fig.subplots(1, len(heatmaps), sharex=True, sharey=True)
# for i, (opt, heatmap) in enumerate(heatmaps.items()):
#     ax = axes[i]
#     sns.heatmap(heatmap, ax=ax, xticklabels=sets, yticklabels=sets)
#     ax.set_title(opt)
# plt.show()

In [ ]:


 
# for use_mean in [False]:
#     aggregate_func = lambda x: x
#     if use_mean:
#         aggregate_func = np.mean

    

#     mean_ranking_full = performance[p_cols].rank(axis=1).apply(aggregate_func)
#     mean_ranking_full = performance[p_cols]
#     mean_ranking_full["problem_id"] = performance["problem_id"] if not use_mean else "mean_problem"
#     mean_ranking_full["set"] = "full"
#     mean_ranking_full["split"] = -1
#     mean_ranking_full["seed"] = -1
#     mean_ranking_full["use_mean"] = use_mean
#     mean_ranking_full = melt(mean_ranking_full)
#     mean_ranking_full = add_performance(mean_ranking_full)
    

#     mean_ranking_dev = subset_dev[p_cols].rank(axis=1).apply(aggregate_func)
#     mean_ranking_dev["problem_id"] = subset_dev["problem_id"] if not use_mean else "mean_problem"
#     mean_ranking_dev["set"] = "discrepancy_dev"
#     mean_ranking_dev["split"] = -1
#     mean_ranking_dev["seed"] = -1
#     mean_ranking_dev["use_mean"] = use_mean
#     mean_ranking_dev = melt(mean_ranking_dev)
#     mean_ranking_dev = add_performance(mean_ranking_dev)

#     mean_ranking_test = subset_test[p_cols].rank(axis=1).apply(aggregate_func)
#     mean_ranking_test["problem_id"] = subset_test["problem_id"] if not use_mean else "mean_problem"
#     mean_ranking_test["set"] = "discrepancy_test"
#     mean_ranking_test["split"] = -1
#     mean_ranking_test["seed"] = -1
#     mean_ranking_test["use_mean"] = use_mean
#     mean_ranking_test = melt(mean_ranking_test)
#     mean_ranking_test = add_performance(mean_ranking_test)

#     # easiest
#     costs = performance[p_cols].values
#     ids = pareto(-costs)
#     pareto_performance = performance.iloc[ids]
#     mean_ranking_pareto_easiest = pareto_performance[p_cols].rank(axis=1).apply(aggregate_func)
#     mean_ranking_pareto_easiest["problem_id"] = pareto_performance["problem_id"] if not use_mean else "mean_problem"
#     mean_ranking_pareto_easiest["set"] = "pareto_full_easiest"
#     mean_ranking_pareto_easiest["split"] = -1
#     mean_ranking_pareto_easiest["seed"] = -1
#     mean_ranking_pareto_easiest["use_mean"] = use_mean
#     mean_ranking_pareto_easiest = melt(mean_ranking_pareto_easiest)
#     mean_ranking_pareto_easiest = add_performance(mean_ranking_pareto_easiest)

#     # hardest
#     costs = performance[p_cols].values
#     ids = pareto(costs)
#     pareto_performance = performance.iloc[ids]
#     mean_ranking_pareto_hardest = pareto_performance[p_cols].rank(axis=1).apply(aggregate_func)
#     mean_ranking_pareto_hardest["problem_id"] = pareto_performance["problem_id"] if not use_mean else "mean_problem"
#     mean_ranking_pareto_hardest["set"] = "pareto_full_hardest"
#     mean_ranking_pareto_hardest["split"] = -1
#     mean_ranking_pareto_hardest["seed"] = -1
#     mean_ranking_pareto_hardest["use_mean"] = use_mean
#     mean_ranking_pareto_hardest = melt(mean_ranking_pareto_hardest)
#     mean_ranking_pareto_hardest = add_performance(mean_ranking_pareto_hardest)



#     ranking = []
#     ranking.append(mean_ranking_full)
#     ranking.append(mean_ranking_dev)
#     ranking.append(mean_ranking_test)
#     ranking.append(mean_ranking_pareto_easiest)
#     ranking.append(mean_ranking_pareto_hardest)

#     n_splits = n_problems // (subset_size * 2)
#     print(n_splits)
#     X = performance.values
#     y = performance["benchmark_id"].to_numpy()

#     split_classes = [StratifiedShuffleSplit, ShuffleSplit]

#     for split_class in split_classes:
#         for seed in seeds:
#             sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.5, random_state=seed)
#             sss.get_n_splits(X, y)
#             for i, (train_index, test_index) in enumerate(sss.split(X, y)):
#                 D_dev = performance.iloc[train_index][p_cols].rank(axis=1).apply(aggregate_func)
#                 D_dev["problem_id"] = performance.iloc[train_index]["problem_id"] if not use_mean else "mean_problem"
#                 D_dev["set"] = f"{split_class.__name__}_dev"
#                 D_dev["split"] = i
#                 D_dev["seed"] = seed
#                 D_dev["use_mean"] = use_mean
#                 D_dev = melt(D_dev)
#                 D_dev = add_performance(D_dev)
                
#                 D_test = performance.iloc[test_index][p_cols].rank(axis=1).apply(aggregate_func)
#                 D_test["problem_id"] = performance.iloc[test_index]["problem_id"] if not use_mean else "mean_problem"
#                 D_test["set"] = f"{split_class.__name__}_test"
#                 D_test["split"] = i
#                 D_test["seed"] = seed
#                 D_test["use_mean"] = use_mean
#                 D_test = melt(D_test)
#                 D_test = add_performance(D_test)
#                 ranking.extend([D_dev, D_test])

#     df_ranking = pd.DataFrame(ranking) if use_mean else pd.concat(ranking).reset_index(drop=True)
#     df_ranking = df_ranking.melt(id_vars=["set", "split", "seed", "problem_id", "use_mean"], value_vars=p_cols, var_name="optimizer_id", value_name="rank")
#     rankings.append(df_ranking)

# df_ranking = pd.concat(rankings).reset_index(drop=True)
# df_ranking
